In [1]:
import os
import json
import uuid
import pandas as pd
import numpy as np


from typing import Dict, List

In [2]:
input_path = 'raw/'  # where the original model.bim and report.json are
output_path = 'curated/'  # where the central file, the new model.bim and the new report.json will be generated

report_name = 'FINMA Report'  # name of the report
original_language = 'DE'  # original languages code

# do not change
language_table = 'LanguageSelection'  # the table holding the different languages code
language_table_column_code = 'LanguageCode'  # the column with the languages code
titles_table = 'TitlesTranslations'  # the table holding the title measures

field_parameters_prefix = 'ZZ FP'  # prefix to use for field parameters, allow the code to seperate translation FP from business FP


# dictionary to hold all the argument
params = {
    'input_path': input_path,
    'output_path': output_path,
    'report_name': report_name,
    'original_language': original_language,
    'field_parameters_prefix': field_parameters_prefix,
    'language_table': language_table,
    'language_table_column_code': language_table_column_code,
    'titles_table': titles_table
}


In [3]:
class TranslationGlobalTools:
    """
    This class is just here to collect all arguments from the classes below
    and 2 functions to open the report.json and model.bim
    """
    def __init__(self, params: Dict[str, str]) -> None:
        self.report_name = params.get('report_name', None)
        self.original_language = params.get('original_language', None)

        self.input_path = params.get('input_path', None) + self.report_name + '/'
        self.output_path = params.get('output_path', None) + self.report_name
        self.machine_input_path = 'dummy-artifacts/'
        
        self.field_parameters_prefix = params.get('field_parameters_prefix', None)

        self.language_table = params.get('language_table', None)
        self.language_table_column_code = params.get('language_table_column_code', None)
        self.titles_table = params.get('titles_table', None)
        
        self.metadata_path = f'{self.output_path}/metadata_new.csv'  # path of metadata and data that would need translations
        self.title_path = f'{self.output_path}/title_new.csv'  # path of titles that would need translations
        self.existing_field_parameters_path = f'{self.output_path}/translation_old.csv'  # path for already existing metadata and data translations
        self.existing_titles_path = f'{self.output_path}/title_old.csv'  # path for already existing title translations
        self.central_file_path = f'{self.output_path}/central.xlsx'  # path where the central file will be saved

        if not os.path.exists(self.output_path):
            os.mkdir(self.output_path)

        print('TranslationGlobalTools initiated.')

    
    def open_report_json(self) -> dict:  # to open the original report.json file
        with open(self.input_path + 'report.json') as f:
            report_json = json.load(f) #dict
        return report_json


    def open_model_bim(self) -> dict:  # to open the original model.bim file
        with open(self.input_path + 'model.bim') as f:
            model_bim = json.load(f) #dict
        return model_bim


In [4]:
class TranslationExtractor(TranslationGlobalTools):
    """
    Goal of this class is:
    -> to collect the measures / columns / titles in need of translations
    -> to collect existing translation of measures / columns / titles
    -> to merge the 2 collections into 1 central file
    """
    def __init__(self, params: Dict[str, str]) -> None:
        super().__init__(params)
        print('TranslationExtractor initiated.')

    def get_list_of_visuals(self) -> list:
        """ 
        Goal of this function is:
        -> to return any visuals that can contains projections or titles

        returns:  a list of visuals
        """
        report_json = self.open_report_json()
        report_pages = report_json['sections']
        visuals_to_extract = []

        for page in report_pages: #give dictionnaries

            page_visuals = page['visualContainers'] # give list

            for visual in page_visuals: # give dictionnaries
                visual_config = json.loads(visual['config']) #Load the config in dictionary
                singleVisual = visual_config.get('singleVisual', False)

                if singleVisual: #Visual group are just group of visual not individual visual
                    if singleVisual.get('projections', False):
                        visuals_to_extract.append(singleVisual)

        return visuals_to_extract


    def extract_metadata(self) -> list:
        """
        This function's goal is to : 
        -> extract all columns and measure that are used at visual level
        i.e. that would need a translation

        returns: list of metadata/data used in the report with the original text
        """
        visuals_to_extract = self.get_list_of_visuals()
        all_used_metadata = []

        for visual in visuals_to_extract:
            visual_prototypequery = visual['prototypeQuery']
            sources_from = visual_prototypequery['From']
            source_dict = dict([(src['Name'], src['Entity']) for src in sources_from])  # the source 

            column_properties_dict = visual.get('columnProperties', dict())

            for sub_el in visual_prototypequery['Select']:

                is_measure = sub_el.get('Measure', False) != False
                is_column = sub_el.get('Column', False) != False
                is_aggregation = sub_el.get('Aggregation', False) != False

                if is_measure:
                    source_key = sub_el['Measure']['Expression']['SourceRef']['Source']
                    source_property = sub_el['Measure']['Property']
                    visual_kind = 'Measure'
                    function_type = ''
                    possibility = 1
                
                if is_column:
                    source_key = sub_el['Column']['Expression']['SourceRef']['Source']
                    source_property = sub_el['Column']['Property']
                    visual_kind = 'Column'
                    function_type = ''
                    possibility = 1

                if is_aggregation:
                    source_key = sub_el['Aggregation']['Expression']['Column']['Expression']['SourceRef']['Source']
                    source_property = sub_el['Aggregation']['Expression']['Column']['Property']
                    visual_kind = 'Aggregation'
                    function_type = sub_el['Aggregation']['Function']
                    possibility = 0

                if is_measure or is_column or is_aggregation:
                    # if the displayName (if it exists) is not the same, use it
                    col_prop_key = source_dict[source_key] + '.' +  source_property 
                    display_name = column_properties_dict.get(col_prop_key, dict()).get('displayName', False)
                    
                    used_name = source_property
                    if display_name != False:
                        used_name = display_name

                    # to understand logic between NativeReferenceName, Property and displayName
                    #if used_name != sub_el['NativeReferenceName'] and possibility:
                    #    print(used_name, '----', display_name, '----', sub_el['NativeReferenceName'])


                    all_used_metadata.append((visual_kind, col_prop_key, used_name, function_type, possibility))
                else:
                    a = visual
                    raise 'Unrecognized Visual Type.'

        return all_used_metadata

    def extract_titles(self) -> list:
        """
        This function's goal is to : 
        -> extract all titles that are used at visual level
        i.e. that would need a translation

        returns: list of title used in the report
        """
        visuals_to_extract = self.get_list_of_visuals()
        info = [visual.get('vcObjects', None) for visual in visuals_to_extract]
        info = [visual.get('title', None) for visual in info if visual]
        titles = [el[0]['properties'] for el in info if el]

        titles_name = []

        for title in titles:
            temp = title.get('text', None)
            if temp is not None:
                title_info = title['text']['expr']
                if title_info.get('Literal', False):
                    title_text = title_info['Literal']['Value']
                    title_text = title_text.strip(title_text[0])
                    titles_name.append(title_text)

        return list(set(titles_name))


    def save_translation_needed(self, replace: bool = False) -> None:
        metadata_list = self.extract_metadata()
        titles_list = self.extract_titles()

        df_metadata = pd.DataFrame(metadata_list, columns=['Type', 'SemanticCompleteName', self.original_language, 'AggregationFunction', 'Possible'])
        df_metadata.drop_duplicates(['Type', 'SemanticCompleteName', self.original_language], inplace=True)
        df_metadata.sort_values(['Type', 'SemanticCompleteName'], inplace=True)
        df_metadata['SemanticEntityName'] = df_metadata['SemanticCompleteName'].map(lambda x: '.'.join(x.split('.')[1:]))
        df_metadata = df_metadata[['Type', 'SemanticCompleteName', 'SemanticEntityName', 'AggregationFunction', 'Possible', self.original_language]]

        df_title = pd.DataFrame({self.original_language: list(set(titles_list))})
        df_title = df_title[df_title[self.original_language].str.strip() != ""]

        error_str = ''

        if not os.path.exists(self.metadata_path) or replace: 
            df_metadata.to_csv(self.metadata_path)
        else:
            error_str += f'\nFile for Metadata already exists in the given path: {self.metadata_path}'

        if not os.path.exists(self.title_path) or replace:
            df_title.to_csv(self.title_path)
        else:
            error_str += f'\nFile for Titles already exists in the given path: {self.title_path}'
        
        if error_str:
            raise Exception(error_str)


    def save_central_file(self, replace: bool=False) -> None:
        
        new_path = self.metadata_path
        df_metadata = pd.read_csv(new_path, index_col=0)

        df_metadata = df_metadata[df_metadata['Possible'] == 1]
        df_metadata.drop(['AggregationFunction', 'Possible'], axis=1, inplace=True)

        df_metadata = df_metadata[['Type', 'SemanticCompleteName', 'SemanticEntityName', self.original_language]]
                
        new_path = self.title_path
        df_titles = pd.read_csv(new_path, index_col=0)

        if not os.path.exists(self.central_file_path) or replace: 
            with pd.ExcelWriter(self.central_file_path) as writer:  
                df_metadata.to_excel(writer, sheet_name='Metadata')
                df_titles.to_excel(writer, sheet_name='Titles')
        else:
            raise Exception(f'\nFile for Central File already exists in the given path: {self.central_file_path}')


    def generate_central_file(self, replace: bool=False) -> None:
        self.save_translation_needed(replace=replace)
        self.save_central_file(replace=replace)


In [5]:
class TranslationEntityCreator(TranslationGlobalTools):
    """
    Goal of this class is:
    -> Open the central translation file
    -> Collect existing field parameters (FP) and relationship in the semantic model
    -> Delete existing FP and relationship

    -> Create the relationships
    -> Create the FP

    Deleting all and remaking all is simplifying the code and allow us to make sure we are always
    with the most up-to-date translation
    """

    def __init__(self, params: Dict[str, str]) -> None:
        super().__init__(params)
        print('TranslationEntityCreator initiated.')


    def create_relationship(self) -> List[Dict[str, str]]:
        """
        This function's goal is:
        -> add missing relationships between FPs' and the language table

        return only relationship with language table
        """
        model = self.open_model_bim()

        df_metadata = pd.read_excel(self.central_file_path, sheet_name='Metadata', index_col=0)
        df_metadata.fillna('', inplace=True)

        new_relationships = []

        for n, semantic_name in df_metadata.iterrows():
            
            row_info = dict(semantic_name)

            type_ = row_info['Type']
            semantic_name = row_info['SemanticCompleteName'].replace('[', '').replace(']', '').replace('.', '_')
            table_name = f"{self.field_parameters_prefix} {semantic_name} {type_}"
                
            new_relationship = {
                                    "name": str(uuid.uuid4()),
                                    "crossFilteringBehavior": "bothDirections",
                                    "fromCardinality": "one",
                                    "fromColumn": self.language_table_column_code,
                                    "fromTable": "LanguageSelection",
                                    "toColumn": self.language_table_column_code,
                                    "toTable": table_name
                                }

            new_relationships.append(new_relationship)

        return new_relationships 


    def create_field_parameters(self) -> List[Dict]:
        """
        This function's goal is:
        -> seperate FPs related to translation work to the other tables
        -> create from scraft the FPs

        return all the table old and new FPs (without the old FP)
        """

        with open(f'{self.machine_input_path}dummy_field_parameters.txt') as f:
            dummy_field_parameter = f.read() #dict

        df_metadata = pd.read_excel(self.central_file_path, sheet_name='Metadata', index_col=0)
        df_metadata.fillna('', inplace=True)

        self.existing_languages = [code for code in list(df_metadata.columns)[3:] if 'semantic' not in code]

        new_field_parameters = []

        # creating one field parameters per line of the dataframe
        for n, semantic_name in df_metadata.iterrows():
            
            row_info = dict(semantic_name)

            type_ = row_info['Type']
            semantic_name = row_info['SemanticCompleteName'].replace('[', '').replace(']', '').replace('.', '_')
            table_full_name = f"{self.field_parameters_prefix} {semantic_name}"

            temp_fp = dummy_field_parameter
            temp_fp = temp_fp.replace('{FULL_TABLE_NAME}', table_full_name)
            temp_fp = temp_fp.replace('{TYPE}', type_)

            temp_fp = temp_fp.replace('{ID_ANO}', str(uuid.uuid4()))
            temp_fp = temp_fp.replace('{ID_TABLE}', str(uuid.uuid4()))
            temp_fp = temp_fp.replace('{ID_COL1}', str(uuid.uuid4()))
            temp_fp = temp_fp.replace('{ID_COL2}', str(uuid.uuid4()))
            temp_fp = temp_fp.replace('{ID_COL3}', str(uuid.uuid4()))
            temp_fp = temp_fp.replace('{ID_COL4}', str(uuid.uuid4()))

            fp_dict = json.loads(temp_fp)

            expression = ['{']

            text_original_language = row_info[self.original_language]

            for n_code, code in enumerate(self.existing_languages):

                text = row_info[code]
                if not text: #If there is no translation available, we want to use the report default text
                    text = text_original_language

                semantic_table = row_info['SemanticCompleteName'].split('.')[0]
                semantic_column = '.'.join(row_info['SemanticCompleteName'].split('.')[1:])
                
                # if a replacement exist 
                if f'semantic{code}' in row_info.keys() and row_info.get(f'semantic{code}', '').strip() != '':
                    semantic_table = row_info[f'semantic{code}' ].split('.')[0]
                    semantic_column = '.'.join(row_info[f'semantic{code}' ].split('.')[1:])
                
                semantic_column = semantic_column.replace(']', ']]') #power bi specific naming convention

                part = f"    (\"{text}\", NAMEOF('{semantic_table}'[{semantic_column}]), {n_code}, \"{code}\"),"

                expression.append(part)

            expression[-1] = expression[-1][:-1] #delete last coma 

            expression.append('}')

            fp_dict['partitions'][0]['source']['expression'] = expression

            new_field_parameters.append(fp_dict)


        return new_field_parameters


    def create_title_translations(self) -> List[Dict]:

        language_table_name = self.language_table
        titles_table = self.titles_table
        column_name = self.language_table_column_code
        switch_formula = f'SWITCH(MIN({language_table_name}[{column_name}]),'
        original_code = self.original_language
    
        with open(f'{self.machine_input_path}title_table.txt') as f:
            translation_table = json.loads(f.read()) #dict

        df_title = pd.read_excel(self.central_file_path, sheet_name='Titles', index_col=0)
        df_title.fillna('', inplace=True)

        new_title_measure = []

        for _, row_info in df_title.iterrows():

            pivot = row_info[original_code]
            measure_name = pivot.replace('[', '').replace(']', '')

            expression = [switch_formula]

            for k, v in row_info.items():
                if k != original_code:  # according to the SWITCH() docs, we don't need the default language as an option, it will be the last argument
                    if v:  # use original text if no translation is available
                        expression.append(f'    "{k}", "{v}",',)
                    else:
                        expression.append(f'    "{k}", "{pivot}",',)

            expression.append(f'    "{pivot}")')

            title_dict ={
                'name': f'{measure_name}_title',
                'expression': expression,
                'lineageTag': str(uuid.uuid4()),
                }

            new_title_measure.append(title_dict)

        translation_table['measures'] = new_title_measure

        return  [translation_table]


    def create_language_table(self) -> List[Dict]:
        
        with open(f'{self.machine_input_path}language_selection.txt') as f:
            language_table = json.loads(f.read()) #dict

        rows = '{{"' + '"}, {"'.join(self.existing_languages) + '"}}'
        language_table['partitions'][0]['source']['expression'] = f'DATATABLE("LanguageCode", STRING, {rows})'
        
        language_table
            
        return [language_table]


    def generate_new_model_bim(self, file_name: str = 'model.bim') -> None:
        """
        This function's goal is to complete the missing relationship between FPs' and the language table
        """
        new_relationships = self.create_relationship()
        field_parameters = self.create_field_parameters()
        titles_translations = self.create_title_translations()
        language_table = self.create_language_table()

        model = self.open_model_bim()

        model['model']['relationships'].extend(new_relationships)
        model['model']['tables'].extend(field_parameters + titles_translations + language_table)

        with open(f'{self.output_path}/{file_name}', 'w', encoding='utf-8') as json_file:
            print(f'{self.output_path}/{file_name}')
            json.dump(model, json_file, indent=4, ensure_ascii=False)


In [6]:
class TranslationEntityConnector(TranslationGlobalTools):
    """
    Goal of this class is:
    -> Open the central translation file
    -> Collect all visual that would need title or metadata translation
    -> Connect the needed field parameters or title measure

    Deleting all and remaking all is simplifying the code and allow us to make sure we are always
    with the most up-to-date translation
    """

    def __init__(self, params: Dict[str, str]) -> None:
        super().__init__(params)
        print('TranslationEntityConnector initiated.')


    def connect_title_to_measure(self) -> Dict:
        report_json = self.open_report_json()
        table_title = self.titles_table
        pages = report_json['sections']

        report_pages_after_change = []

        for n_page, page in enumerate(pages):
            
            visuals = page['visualContainers']
            visuals_after_change = []
            
            for n_visual, visual in enumerate(visuals):
                
                visual_config = json.loads(visual['config'])
                
                if 'singleVisual' in visual_config.keys(): 
                    #singleVisual needed
                    singleVisual = visual_config['singleVisual']
                else:
                    visuals_after_change.append(visual)
                    continue
                
                if not singleVisual.get('projections', False):
                    visuals_after_change.append(visual)
                    continue
                
                if singleVisual.get('vcObjects', False) and singleVisual['vcObjects'].get('title', False): 
                    #if there is not title, we can leave this visual
                    title = singleVisual['vcObjects'].get('title')[0]['properties']
                else:
                    visuals_after_change.append(visual)
                    continue

                if title.get('text', False) and title['text']['expr'].get('Literal', False):  
                    #if this a hand written value
                    title_info = title['text']['expr']
                else:
                    visuals_after_change.append(visual)
                    continue

                title_value = title_info['Literal']['Value']
                title_value = title_value.replace(title_value[0], '')
                if not title_value.strip():
                    visuals_after_change.append(visual)
                    continue

                measure_name = title_value.replace('[', '').replace(']', '') + '_title' #the name of the measure
                measured_expression = {'Expression':{'SourceRef':{'Entity': table_title}},'Property': measure_name} #the right json format
                
                title['text']['expr'].pop('Literal') # take the old version out
                title['text']['expr']['Measure'] = measured_expression # add the measure connection

                singleVisual['vcObjects'].get('title')[0]['properties'] = title
                visual_config['singleVisual'] = singleVisual

                visual['config'] = json.dumps(visual_config, ensure_ascii=False)

                visuals_after_change.append(visual)

            page['visualContainers'] = visuals_after_change
            report_pages_after_change.append(page)

        report_json['sections'] = report_pages_after_change

        return report_json

    def _add_field_parameter_to_visual(self, prototypeQuery: Dict, projections: Dict, column_properties_dict: Dict, fp_type_dict: Dict) -> Dict:

        source_tables_dict = dict({(el['Name'], el['Entity']) for el in prototypeQuery['From']})

        # every reference name are not the semantic name but a simplified version of it
        # build a dict to get the original version from the simplified one
        projections_table_dict = {}

        for el in prototypeQuery['Select']:
            type_key = list(el.keys())[0]
            if type_key != 'Aggregation': # Aggregation are not supported on field parameters
                source_table = el[type_key]['Expression']['SourceRef']['Source']
                source_column = el[type_key]['Property']
                used_name = el['Name']
                projections_table_dict[used_name] = (source_tables_dict[source_table], source_column)

        # create the field-parameters dictionnary
        queryFieldParametersByRole = {}

        # For every dimension on the projections
        for key in projections.keys():

            fp_mapping = []

            # for every type of projections i.e. X, Y and tooltips
            for n, el in enumerate(projections[key]):

                table, column = projections_table_dict.get(el['queryRef'], ('', ''))
                semantic_name = table + '.' + column
                displayName = column_properties_dict.get(semantic_name, dict()).get('displayName', False)

                used_name = column
                if displayName != False:
                    used_name = displayName

                # if the projection used as a field parameters equivalent
                if fp_type_dict.get(semantic_name, False):
                    type_key = fp_type_dict[semantic_name]
                    
                    table_name = semantic_name.replace('[', '').replace(']', '').replace('.', '_')
                    new_mapping = {
                        'index': n, 
                        'length': 1, 
                        'expr': 
                            {'Column': 
                                {'Expression': 
                                    {
                                        'SourceRef': 
                                            {'Entity': f'{self.field_parameters_prefix} {table_name} {type_key}'}
                                        }, 
                                        'Property': f'{self.field_parameters_prefix} {table_name} {type_key}'}
                                        }
                                        
                            }

                    fp_mapping.append(new_mapping)

            queryFieldParametersByRole[key] = fp_mapping

        return queryFieldParametersByRole # we return the formatted field parameter hashmap


    def connect_translations_to_fp(self, report_json: Dict) -> Dict:
        df_metadata = pd.read_excel(self.central_file_path, sheet_name='Metadata', index_col=0)
        df_metadata.fillna('', inplace=True)

        pages = report_json['sections']

        df_metadata['FP_KEY'] = df_metadata['SemanticCompleteName'] + '_' + df_metadata[self.original_language]

        fp_type_dict = dict(zip(df_metadata['SemanticCompleteName'].tolist(), df_metadata['Type'].tolist())) 

        report_pages_after_change = []

        for n_page, page in enumerate(pages):

            visuals = page['visualContainers']
            visuals_after_change = []
            
            for n_visual, visual in enumerate(visuals):
                
                visual_config = json.loads(visual['config'])
                
                if 'singleVisual' in visual_config.keys():  #singleVisual needed
                    singleVisual = visual_config['singleVisual']
                else:
                    visuals_after_change.append(visual)
                    continue
                
                if singleVisual.get('projections', False): 
                    
                    prototypeQuery = singleVisual['prototypeQuery']
                    projections = singleVisual['projections']
                else:
                    visuals_after_change.append(visual)
                    continue

                # if the field parameters already exist we leave
                if singleVisual.get('queryFieldParametersByRole', False):
                    visuals_after_change.append(visual)
                    continue

                column_properties_dict = singleVisual.get('columnProperties', dict())

                queryFieldParametersByRole = self._add_field_parameter_to_visual(prototypeQuery, projections, column_properties_dict, fp_type_dict)
                visual_config['singleVisual']['queryFieldParametersByRole'] = queryFieldParametersByRole
                visual['config'] = json.dumps(visual_config, ensure_ascii=False)

                visuals_after_change.append(visual)

            page['visualContainers'] = visuals_after_change
            report_pages_after_change.append(page)

        report_json['sections'] = report_pages_after_change

        return report_json


    def create_if_need_filter_language(self, report_json: Dict) -> Dict:
        
        filters_text = report_json.get('filters', '')

        new_filter = {
                    'name': 'Filtere324fc6402c259d3cfd5',
                    'expression': {'Column': {'Expression': {'SourceRef': {'Entity': 'LanguageSelection'}},
                    'Property': 'LanguageCode'}},
                    'filter': {'Version': 2,
                    'From': [{'Name': 'l', 'Entity': 'LanguageSelection', 'Type': 0}],
                    'Where': [{'Condition': {'In': {'Expressions': [{'Column': {'Expression': {'SourceRef': {'Source': 'l'}},
                            'Property': 'LanguageCode'}}],
                        'Values': [[{'Literal': {'Value': f"'{self.original_language}'"}}]]}}}]},
                    'type': 'Categorical',
                    'howCreated': 1,
                    'objects': {'general': [{'properties': {'requireSingleSelect': {'expr': {'Literal': {'Value': 'true'}}}}}]}
                        }

        if filters_text:

            filters = json.loads(filters_text)

            for filter_ in filters:
                name = filter_.get('expression', dict()).get('Column', dict()).get('Expression', dict()).get('SourceRef', dict()).get('Entity', "")
                if self.language_table == name:
                    return report_json

            filters.append(new_filter)
        else:
            filters = [new_filter]


        filters_text = json.dumps(filters, ensure_ascii=False)
        report_json['filters'] = filters_text

        return report_json


    def create_new_report_json(self, file_name: str = 'report.json') -> None:
        
        report_json = self.connect_title_to_measure()
        report_json = self.connect_translations_to_fp(report_json)
        report_json = self.create_if_need_filter_language(report_json)
        
        with open(f'{self.output_path}/{file_name}', 'w', encoding='utf-8') as json_file:
            print(f'{self.output_path}/{file_name}')
            json.dump(report_json, json_file, indent=4, ensure_ascii=False)

In [7]:
class Translator(TranslationExtractor, TranslationEntityCreator, TranslationEntityConnector):
    def __init__(self, params: Dict[str, str]) -> None:
        super().__init__(params)
        print('Everything initiated.')

In [8]:
translator = Translator(params)

TranslationGlobalTools initiated.
TranslationEntityConnector initiated.
TranslationEntityCreator initiated.
TranslationExtractor initiated.
Everything initiated.


In [9]:
NEED_CENTRAL = False

if NEED_CENTRAL:
    translator.generate_central_file(replace=True)

In [10]:
NEED_MODEL = True

if NEED_MODEL:
    translator.generate_new_model_bim()

curated/FINMA Report/model.bim


In [11]:
NEW_REPORT = True

if NEW_REPORT:
    translator.create_new_report_json()

curated/FINMA Report/report.json
